In [1]:
# !pip install aiohttp

In [2]:
import aiohttp
import asyncio
import requests
import numpy as np
import pandas as pd

import warnings as wn
wn.filterwarnings("ignore")

In [3]:
headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.2 Safari/605.1.15"
    }

In [4]:
# 1 - ROIC (10Y Median) -> Higher than 12% OR ROE (10Y Median) -> Higher than 15%
# 2 - P/E Ratio -> Lower than a Sector/Industry Median)
# 3 - EBIT/EV Ratio (1.0 / (EV/EBIT)) -> Higher than a risk-free rate (13 Weeks T-Bills))

In [5]:
def get_dataframe(data):
    cols, rows = data["headers"], data["rows"]
    df = pd.DataFrame(data=rows, columns=list(cols.keys()))
    df.columns = list(cols.values())
    df.set_index(df.columns[0], inplace=True)
    return df


def get_response(url):
    session = requests.Session()
    response = session.get(url=url, headers=headers)
    return response.json()


def get_tickers():
    url = f"https://api.nasdaq.com/api/screener/stocks?limit=0&marketcap=mega|large"
    session = requests.Session()
    response = session.get(url=url, headers=headers)
    data = response.json()
    df = get_dataframe(data["data"]["table"])
    if isinstance(df, pd.DataFrame):
        df.index = [i.replace("/", ".") for i in df.index]
    return df.index.tolist()


def get_risk_free_rate():
    url = "https://query1.finance.yahoo.com/v8/finance/chart/^IRX"
    response = get_response(url)
    return response["chart"]["result"][0]["meta"]["regularMarketPrice"] / 100


In [6]:
def get_profile(ticker, risk_free_rate, data):
    try:
        metadata = data["datasets"]["metadata"]
        if metadata["template_type"] == "normal":
            roic_10y_median = np.median([i[1] for i in data["datasets"]["chart"][-10:]])
            lbls = ["name", "exchange", "sector", "industry", "pe", "sector_pe_median", "industry_pe_median"]
            rows = [metadata[i] for i in lbls]
            rows[4:] = [float(i) for i in rows[4:]]
            lbls.extend(["roic_10y_median", "ebit_ev"])
            rows.extend([roic_10y_median, 1.0 / float(metadata["ev_ebit"])])
            df = pd.DataFrame(data=rows, index=lbls, columns=[ticker])
            return df
    except (KeyError, ValueError, ZeroDivisionError):
        pass


async def get_async_response(ticker, risk_free_rate, session):
    url = f"https://api.quickfs.net/stocks/{ticker}/ovr/Annual/?sortOrder=ASC"
    async with session.get(url, headers=headers) as response:
        data = await response.json()
        return get_profile(ticker, risk_free_rate, data)


async def main(tickers):
    risk_free_rate = get_risk_free_rate()
    async with aiohttp.ClientSession() as session:
        return pd.concat(
            await asyncio.gather(*[asyncio.create_task(get_async_response(ticker, risk_free_rate, session)) for ticker in tickers]),
            axis=1
        ).T

In [7]:
tickers = get_tickers()
df = await main(tickers)
df

,name,exchange,sector,industry,pe,sector_pe_median,industry_pe_median,roic_10y_median,ebit_ev
AAPL,Apple Inc,NASDAQ,Information Technology,"Technology Hardware, Storage & Peripherals",31.4,37.3,27.6,0.280653,0.037453
MSFT,Microsoft Corporation,NASDAQ,Information Technology,Software,35.9,37.3,37.3,0.220318,0.035088
GOOG,Alphabet Inc Class C,NASDAQ,Communication Services,Interactive Media & Services,25.6,30.0,30.0,0.161311,0.051546
GOOGL,Alphabet Inc Class A,NASDAQ,Communication Services,Interactive Media & Services,25.4,30.0,30.0,0.161311,0.052083
AMZN,Amazon.com Inc,NASDAQ,Consumer Discretionary,Internet & Direct Marketing Retail,307.7,34.0,58.6,0.068497,0.010081
...,...,...,...,...,...,...,...,...,...
USFD,US Foods Holding Corp,NYSE,Consumer Staples,Food & Staples Retailing,29.3,27.0,39.0,0.026179,0.051546
TPR,Tapestry Inc,NYSE,Consumer Discretionary,"Textiles, Apparel & Luxury Goods",11.4,34.0,34.0,0.134088,0.102041
SRPT,Sarepta Therapeutics Inc,NASDAQ,Health Care,Biotechnology,-9.0,29.0,27.1,-0.411871,-0.0625
KEP,Korea Electric Power Corp ADR,NYSE,Utilities,Electric Utilities,-0.6,23.2,23.8,0.009427,-3.333333


In [8]:
df[(df['roic_10y_median'] > 0.2) & (df['ebit_ev'] > 0.04) & (df['pe'] < df['industry_pe_median'])]

,name,exchange,sector,industry,pe,sector_pe_median,industry_pe_median,roic_10y_median,ebit_ev
TSM,Taiwan Semiconductor Manufacturing,NYSE,Information Technology,Semiconductors & Semiconductor Equipment,16.4,37.3,33.5,0.218247,0.072464
V,Visa Inc. Class A,NYSE,Information Technology,IT Services,31.9,37.3,46.7,0.201404,0.040816
HD,Home Depot Inc,NYSE,Consumer Discretionary,Specialty Retail,18.8,34.0,24.3,0.315162,0.066667
ACN,Accenture plc,NYSE,Information Technology,IT Services,27.8,37.3,46.7,0.369086,0.048077
TXN,Texas Instruments Incorporated,NASDAQ,Information Technology,Semiconductors & Semiconductor Equipment,19.7,37.3,33.5,0.302026,0.05988
UPS,United Parcel Service Inc,NYSE,Industrials,Air Freight & Logistics,14.8,26.4,27.8,0.21968,0.072993
QCOM,Qualcomm Incorporated,NASDAQ,Information Technology,Semiconductors & Semiconductor Equipment,12.5,37.3,33.5,0.204263,0.09009
LMT,Lockheed Martin Corporation,NYSE,Industrials,Aerospace & Defense,20.8,26.4,32.0,0.325688,0.064516
AMAT,Applied Materials Inc,NASDAQ,Information Technology,Semiconductors & Semiconductor Equipment,17.8,37.3,33.5,0.218698,0.067114
BTI,British American Tobacco PLC ADR,NYSE,Consumer Staples,Tobacco,9.2,27.0,17.3,0.273541,0.15625


In [9]:
df1 = df.copy()
df1 = df1[df1["pe"] >= 0]
df1["pe_sector_ratio"] = df1["pe"] / df1["sector_pe_median"]
df1["pe_industry_ratio"] = df1["pe"] / df1["industry_pe_median"]
df1["roic_score"] = df1["roic_10y_median"].rank(method="first", ascending=False)
df1["ebit_ev_score"] = df1["ebit_ev"].rank(method="first", ascending=False)
df1["pe_sector_score"] = df1["pe_sector_ratio"].rank(method="first")
df1["pe_industry_score"] = df1["pe_industry_ratio"].rank(method="first")
df1["total_score"] = 0.5 * df1["roic_score"] + 0.25 * df1["ebit_ev_score"] + 0.1 * df1["pe_sector_score"] + 0.15 * df1["pe_industry_score"]

In [10]:
df1["rank"] = df1["total_score"].rank(method="first")

In [11]:
df1.sort_values("rank").to_csv("magic_formula.csv", encoding="utf-8-sig")